In [11]:
import gradio as gr
import dill
import pandas as pd
import xgboost as xgb
import numpy as np
import pickle

In [12]:
def decode_file(file_path):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
    return obj

model = decode_file('../model/model.pkl')

In [13]:
def predict(gender, age, hypertension, ever_married, work_type, heart_disease, avg_glucose_level, bmi, smoking_status, Residence_type):
    # Mapping for categorical variables
    gender_mapping = {'Male': 1, 'Female': 0}
    hypertension_mapping = {'Yes': 1, 'No': 0}
    ever_married_mapping = {'Yes': 1, 'No': 0}
    work_type_mapping = {'Private': 2, 'Self-employed': 4, 'Govt_job': 3, 'Children': 1, 'Never_worked': 0}
    heart_disease_mapping = {'Yes': 1, 'No': 0}
    smoking_status_mapping = {'formerly smoked': 3, 'Smokes': 1, 'Never smoked': 2, 'Unknown': 0}
    Residence_type_mapping = {'Urban': 1, 'Rural': 0}

    # Map categorical variables to their corresponding numerical values
    gender = gender_mapping[gender]
    hypertension = hypertension_mapping[hypertension]
    ever_married = ever_married_mapping[ever_married]
    work_type = work_type_mapping[work_type]
    heart_disease = heart_disease_mapping[heart_disease]
    smoking_status = smoking_status_mapping[smoking_status]
    Residence_type = Residence_type_mapping[Residence_type]

    inputs = [gender, age, hypertension, ever_married, work_type, heart_disease, avg_glucose_level, bmi, smoking_status, Residence_type]
    input_labels = ['gender', 'age', 'hypertension', 'ever_married', 'work_type', 'heart_disease', 'avg_glucose_level', 'bmi', 'smoking_status', 'Residence_type']

    # Convert the input into a pandas DataFrame
    input_df = pd.DataFrame([inputs], columns=input_labels)

    # Predict the stroke probability
    prediction = model.predict_proba(input_df)[0][1]

    # Return the prediction
    result = "The probability of stroke is {:.2f}%".format(prediction * 100)  # to give a percentage
    return result


In [14]:
predict('Female',57.0,'No',"Yes",'Private',"No",82.54,33.4,'Unknown','Rural')


'The probability of stroke is 3.97%'

In [15]:
input_labels = [
    'gender', 'age', 'hypertension', 'ever_married', 'work_type', 
    'heart_disease', 'avg_glucose_level', 'bmi', 'smoking_status', 'Residence_type'
]
# Create the Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.components.Radio(choices=['Female', 'Male'], label="Gender"),
        gr.components.Slider(label="Age"),
        gr.components.Radio(choices=['Yes', 'No'], label="Hypertension"),
        gr.components.Radio(choices=['Yes', 'No'], label="Ever Married"),
        gr.components.Radio(choices=['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked'], label="Work Type"),
        gr.components.Radio(choices=['Yes', 'No'], label="Heart Disease"),
        gr.components.Number(label="Average Glucose Level"),
        gr.components.Slider(label="BMI"),
        gr.components.Radio(choices=['formerly smoked', 'never smoked', 'smokes', 'Unknown'], label="Smoking Status"),
        gr.components.Radio(choices=['Urban', 'Rural'], label="Residence Type")
    ],
    outputs='text',
    title='Stroke Probability Predictor',
    description='Predicts the probability of having a stroke based on input features.'
)


iface.launch(share = True)


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://5d9faa6049b9c12100.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
